# Notebook to access to data on CCDVisits for the System-level Science Performance Verification Sprint February 3-5 2025

## LSSTComCam ZERO-Point uniformity in Tracts in the selected band in a selected range of dates

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creation date : 2025-02-13
- last update : 2025-02-15
- Redo Visits like here : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/Visits/stat_on_visits_LSSTComCamSim.ipynb
- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)
from astropy.modeling import models, fitting

In [ ]:
import seaborn as sns
from itertools import cycle, islice

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta


In [ ]:
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit

In [ ]:
# Set plotting defaults
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors
from matplotlib import cm

zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})

props = dict(boxstyle='round', facecolor="white", alpha=0.1)

### Definition of functions

In [ ]:
def angle_in_range(alpha, lower, upper):
    return (alpha - lower) % 360 <= (upper - lower) % 360

In [ ]:
def get_bbox_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    ramin,ramax,decmin,decmax in decimal degrees
    """

    xmin = bbox.beginX
    xmax = bbox.endX
    ymin = bbox.beginY
    ymax = bbox.endY
    
    radec_ll = wcs.pixelToSky(xmin, ymin)
    radec_ur = wcs.pixelToSky(xmax, ymax)
        
    return radec_ur.getRa().asDegrees(),radec_ll.getRa().asDegrees(), radec_ll.getDec().asDegrees(),radec_ur.getDec().asDegrees() 


In [ ]:
def isradec_inbbox_angularsep(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    float in degree
    
    """

   
    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)
  
    RAMEAN = np.mean([RAMIN,RAMAX])
    DECMEAN = np.mean([DECMIN,DECMAX])

    sp0 = SpherePoint(longitude=geom.Angle(ra,geom.degrees),latitude=geom.Angle(dec,geom.degrees))
    sp1 = SpherePoint(longitude=geom.Angle(RAMEAN,geom.degrees),latitude=geom.Angle(DECMEAN,geom.degrees)) 
    
    sep = sp0.separation(sp1).asDegrees()
     
    return sep


In [ ]:
def isradec_inbbox_radec(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """

    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)

    flag_ra = angle_in_range(ra,RAMIN,RAMAX)
    flag_dec = angle_in_range(dec,DECMIN,DECMAX)

    flag = flag_ra and flag_dec
    return flag


In [ ]:
def FindTractAndPatch(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    """
     
    try:
        ra = row["ra"]
        dec = row["dec"]
        selectFlag = False
        # loop on tract
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
            wcs=tractInfo.getWcs()
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                    flag = isradec_inbbox_radec(ra,dec,wcs, ibb)
                    selectFlag =  selectFlag or flag
                    if selectFlag:
                        return pd.Series([tractID,patchID])
        return pd.Series([0,0])           
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def FindTractAndPatchFromAngularSep(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    Need the tractsId_list not to scan the whole skymap
    """

     
    try:
        ra = row["ra"]
        dec = row["dec"]
        
        # loop on tracts
        List_of_tractids = []
        List_of_patchids = []
        List_of_sep = []
        
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
         
            wcs=tractInfo.getWcs()
            # loop on patches
            patches_ids = []
            patches_sep = []
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                  
                    sep = isradec_inbbox_angularsep(ra,dec,wcs, ibb)
                    patches_ids.append(patchID)
                    patches_sep.append(sep)
                    
            # find the patch with the minimum distance
            patches_ids=np.array(patches_ids)
            patches_sep= np.array(patches_sep)
            idx_sepmin = int(np.where(patches_sep==patches_sep.min())[0])
            
            
            List_of_tractids.append(tractID)
            List_of_patchids.append(patches_ids[idx_sepmin])
            List_of_sep.append(patches_sep[idx_sepmin])
            
        List_of_tractids=np.array(List_of_tractids)
        List_of_patchids=np.array(List_of_patchids)
        List_of_sep= np.array(List_of_sep)
        idx_sepmin = int(np.where(List_of_sep==List_of_sep.min())[0])

        tractID_sel = List_of_tractids[idx_sepmin]
        patchID_sel = List_of_patchids[idx_sepmin]
       
            
        return pd.Series([tractID_sel,patchID_sel])
               
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def plotAVisit(row,ax=None):
    """
    row : dataframe row
    ax 
    """

    if ax == None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
        
    visitid = row["visitId"]
    ra= row["ra"]
    dec = row["dec"]
    tract_id  = row["tractID"]
    patch_id  = row["patchID"]
    label = f"tract {tract_id} , patch {patch_id}"              
    
    tractInfo = skymap.generateTract(tract_id)
    patches_selected = [patch.getSequentialIndex() for patch in tractInfo]     
    wcs=tractInfo.getWcs()
            
    # loop on patches
    all_ra = []
    all_dec = []
    corners = []
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID == patch_id :
            ibb=patch.getInnerBBox()
            RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, ibb)
            for icorn,corner in enumerate(ibb.getCorners()):
                p = geom.Point2D(corner.getX(), corner.getY())
                coord = wcs.pixelToSky(p)
                corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
                all_ra.append(coord.getRa().asDegrees()) 
                all_dec.append(coord.getDec().asDegrees()) 
    if (len(all_ra)>0) and (len(all_dec)>0):
        all_ra.append(all_ra[0])
        all_dec.append(all_dec[0])
        print(ax)
        ax.plot(all_ra,all_dec,'b-',lw=3,label=label)
        ax.scatter([ra],[dec],marker = 'o',s=20,c="r",label=visitid)
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.legend()
                



## Start Here with Configuration


- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
# where are stored the figures
pathfigs = "figsUniformityZPTracts"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".pdf"

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
band = "z"
#STARTDATE = "20241201"
STARTDATE = "20241101"
where_clause = "instrument = \'" + instrument+ "\'" + "AND band =  \'" + band + "\'" +  "AND exposure.day_obs >= " + f"{STARTDATE}" 
collectionStr = collection.replace("/", "_")
NDET = 9
MAGCUT = 18.0
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"

In [ ]:
# Select the aperture radius
rap = "_35_0_"

# instrumental flux (ADU or photons)
calibFluxStr = f"apFlux{rap}instFlux"
calibFluxErrStr = f"apFlux{rap}instFluxErr"
calibFluxMagStr = f"apFlux{rap}instMag"
calibFluxMagErrStr = f"apFlux{rap}instMagErr"

# flux in nJ or Mag
calibFluxCalStr = f"apFlux{rap}calFlux"
calibFluxCalErrStr = f"apFlux{rap}calFluxErr"
calibFluxCalMagStr = f"apFlux{rap}calMag"
calibFluxCalMagErrStr = f"apFlux{rap}calMagErr"

## Initialisation

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
# Check here the collections available
#for _ in registry.queryCollections():
#    if "LSSTComCam/runs/DRP/DP1/w_2025_05" in _:
#        print(_)

In [ ]:
# List a number of usefull data-product to explore
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name) )
            ):
                print(datasetType)

### Get list of Tracts and Patches

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

In [ ]:
dict_tractpatches = {}
for ref in therefs:
    tract = ref.dataId["tract"]
    table = butler.get(ref) 
    list_of_patches = table['patch'].unique()
    dict_tractpatches[tract] = list_of_patches

### Get CCD Visits

In [ ]:
datasettype = "ccdVisitTable"
ccd_visit_table = butler.get(datasettype,collections=collection)

In [ ]:
# Compute nightobs
ccd_visit_table["nightObs"] = ccd_visit_table.apply(lambda x: x['visitId']//100_000, axis=1)

In [ ]:
#  compute time
ccd_visit_table["Time"] = pd.to_datetime(ccd_visit_table['obsStart'])

In [ ]:
# get airmass
ccd_visit_table["airmass"] = ccd_visit_table["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))

In [ ]:
# Short visit ID
ccd_visit_table["ccdVid"] = ccd_visit_table.index -  ccd_visit_table.index[0]
ccd_visit_table["Vid"] = ccd_visit_table["visitId"]  -   ccd_visit_table["visitId"].iloc[0]

In [ ]:
ccd_visit_table

### Associate Tract and Patch for each visit

To associate Tract and Patch to every visit is a long procedure. This association is done at the first pass time in this notebook
and written into a csv file. At next execution, this visitid-tract-patch is read into a pandas dtaframe. The association is done
on a pair of columns ((visitId,detector).

In [ ]:
# to speed up the calculation of tractID, patchID per visit, this file is saved
# or read back
if os.path.isfile(fn_ccdVisit_tracts_patches):
    print(f"{fn_ccdVisit_tracts_patches} found !!! ==> Read it !!!" )
    ccd_visit_tract_patch_table = pd.read_csv(fn_ccdVisit_tracts_patches,index_col=0) 
    # need to add only "tractID","patchID" to ccd_visit_table
    ccd_visit_tract_patch_table_only = ccd_visit_tract_patch_table[["visitId","detector","tractID","patchID"]]
    # do the merging on the columns (visitId,detector), does not add suffixes to column names
    new_df = pd.merge(
    left=ccd_visit_table, 
    right=ccd_visit_tract_patch_table_only,
    how='left',
    left_on=['visitId', 'detector'],
    right_on=['visitId', 'detector'], suffixes=('', ''))
    #Overwrite the ccd_visit_table by the merge result
    new_df
    ccd_visit_table=new_df
else:
    print(f"{fn_ccdVisit_tracts_patches} NOT found !!! ==> Create it !!! " )
    ccd_visit_table[["tractID","patchID"]] = ccd_visit_table.apply(FindTractAndPatch, axis=1,result_type ='expand')
    # check if some (tract,patch) search failed
    ccd_visit_table_patchnotfound = ccd_visit_table[ccd_visit_table.patchID==0]
    if len(ccd_visit_table_patchnotfound)>0:
        ccd_visit_table_patchnotfound.drop(columns=["tractID","patchID"],inplace=True)
        ccd_visit_table_patchnotfound[["tractID","patchID"]] = ccd_visit_table_patchnotfound.apply(FindTractAndPatchFromAngularSep, axis=1,result_type ='expand')
        for visitindex, row in ccd_visit_table_patchnotfound.iterrows(): 
            ccd_visit_table.loc[visitindex,["tractID","patchID"]] = ccd_visit_table_patchnotfound.loc[visitindex,["tractID","patchID"]]
    #save a subsample for the visit 
    columns_selected = ["visitId","band","detector","ra","dec","llcra","llcdec","ulcra","ulcdec","urcra","urcdec","lrcra","lrcdec","ccdVid","Vid","nightObs","tractID","patchID","zeroPoint","airmass","skyBg","skyNoise","expTime"]
    ccd_visit_tract_patch_table = ccd_visit_table[columns_selected]
    ccd_visit_tract_patch_table.to_csv(fn_ccdVisit_tracts_patches) 


In [ ]:
ccd_visit_table.head()

### Plot One visit

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
plotAVisit(ccd_visit_table.iloc[0],ax=ax)
ax.set_aspect('equal')
ax.set_title(f"{instrument} \n {collectionStr}",fontsize=12)

### sourceTable_visit

In [ ]:
FLAG_ISOLATED_STAR_SOURCES = False
FLAG_SOURCES_TABLE_VISIT = True
FLAG_OBJECTS_TABLE_TRACT = True
FLAG_FGCM = False
FLAG_FGCM_CYCLE5 = True
FLAG_TRANSMISSION = True

In [ ]:
# Try to get the Schema

if FLAG_SOURCES_TABLE_VISIT:

    data_product = "sourceTable_visit"
    #datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= where_clause)
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection,where = where_clause)
    for i, ref in enumerate(datasetRefs):
        print(i,ref.dataId)
        if i>20:
            break
    

In [ ]:
all_sourceTable_visit = []
count=0
for i, ref in enumerate(datasetRefs):
    # reduce used memory by filtering useless
    t = butler.get(ref)

    ## apply selection
    t["psfMag"] = (t["psfFlux"].values*u.nJy).to(u.ABmag).value
    selection_cut = (t.parentSourceId == 0) & (t.sky_source == False) &  (t.detect_isPrimary == True) & (t.extendedness_flag == False) & (t.blendedness_flag==False) & (t.psfMag<MAGCUT)
    tf = t[selection_cut]
    
    n = len(tf)
    count+=n
    if i%100==0:
        print(f"loop {i}, n= +{n}/{count}")

    ## save  
    if n>0:
        all_sourceTable_visit.append(tf)
    

In [ ]:
df = pd.concat(all_sourceTable_visit)

In [ ]:
selection_cut = (df.parentSourceId == 0) & (df.sky_source == False) &  (df.detect_isPrimary == True) & (df.extendedness_flag == False) & (df.blendedness_flag==False)

In [ ]:
df = df[selection_cut]

In [ ]:
df

### Merge the sourceTable with the CCD visit Table

In [ ]:
df_m =  pd.merge(left=df, right=ccd_visit_table,how='left',left_on=['visit', 'detector'],right_on=['visitId', 'detector'], suffixes=('', '_vis'))

In [ ]:
df_m

In [ ]:
cmap = ListedColormap(sns.color_palette("hls", NDET))
all_det_colors = [cmap.colors[idx] for idx in range(NDET)]

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,16),layout="constrained")
axs = axes.flatten()

for idet, ax in enumerate(axs):
    df_m[df_m.detector==idet].plot.scatter(x="x",y="y", marker=".",color=all_det_colors[idet],ax=ax,alpha=0.1)
    ax.set_aspect('equal')
    ax.set_title(f"det = {idet}")
  
    
#plt.gca().set_aspect('equal')
plt.suptitle(f"{instrument} (band {band}) \n {collectionStr}",fontsize=12)

### Calculate Magnitudes

In [ ]:
# Add columns into df_m table to have them all in one place
df_m["psfSn"] = df_m["psfFlux"]/df_m["psfFluxErr"]
df_m["psfMag"] = (df_m["psfFlux"].values*u.nJy).to(u.ABmag).value
df_m["psfMagErr"] = 2.5/np.log(10.0)*(df_m["psfFluxErr"].values/df_m["psfFlux"].values)


# This is the way to apply aperture corrections :
df_m[calibFluxCalStr] = df_m[calibFluxStr]*df_m["localPhotoCalib"]
df_m[calibFluxCalErrStr] = df_m[calibFluxErrStr]*df_m["localPhotoCalib"]


df_m[calibFluxCalMagStr] = (df_m[calibFluxCalStr].values*u.nJy).to(u.ABmag).value
df_m[calibFluxCalMagErrStr] = 2.5/np.log(10.0)*(df_m[calibFluxCalErrStr].values/df_m[calibFluxCalStr].values)

# NOTE: psfFlux is the fgcm calibrated flux.  I'm pretty sure you get the "instrumental" 
# flux by dividing psfFlux by the localPhotoCalib value.
df_m["psfInstMag"] = ((df_m["psfFlux"].values/df_m["localPhotoCalib"].values)*u.nJy).to(u.ABmag).value
df_m["psfGausFluxRatio"] = df_m["psfFlux"]/df_m["gaussianFlux"]

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,16),layout="constrained",sharex=True,sharey=True)
axs = axes.flatten()

for idet, ax in enumerate(axs):
    leg = ax.get_legend()
    df_m[df_m.detector==idet].plot.scatter(x=calibFluxCalMagStr,y=calibFluxCalMagErrStr, marker=".",color=all_det_colors[idet],ax=ax,alpha=0.5,legend=leg,label="aperture Flux")
    df_m[df_m.detector==idet].plot.scatter(x="psfMag",y="psfMagErr", marker="+",color=all_det_colors[idet],ax=ax,alpha=0.5,legend=leg,label="psf Flux")
    ax.set_aspect('auto')
    ax.set_title(f"det = {idet}")
    ax.set_ylim(0.,0.5)
    ax.grid(True)
    #ax.legend()
  
    
#plt.gca().set_aspect('equal')
plt.suptitle(f"{instrument} (band {band})\n {collectionStr}",fontsize=12)

## Compute zero-point difference

$$
\Delta ZP
$$

- difference betwen the local zero-point at the source position relative to the CCD zero-point

In [ ]:
df_m["localPhotoCalibMag"] = (df_m["localPhotoCalib"].values*u.nJy).to(u.ABmag).value
df_m["localPhotoCalibMagErr"] =   2.5/np.log(10.0)*df_m["localPhotoCalibErr"].values/df_m["localPhotoCalib"].values
df_m["DZP"] = df_m["localPhotoCalibMag"] - df_m["zeroPoint"]
df_m["DZPmmag"] = df_m["DZP"]*1000.

#### Parameters for histograms on $\Delta ZP$

In [ ]:
NBINS = 100
if band in ["g","r"]:
    RANGE = (-15,15)
elif band in ["i", "z"]:
    RANGE = (-20,20)
else:
    RANGE = (-30,30)
xfit = np.linspace(RANGE[0],RANGE[1],NBINS)
AMAX = 5000.

#### All Combined DZP histogram, stat and fit

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained",sharex=True,sharey=True)

leg = ax.get_legend()
stats = df_m["DZPmmag"].agg(["mean","median","std"])
label =  "$\overline{m}= $" + f"{stats[0]:.2f} mmag \n" + " $\sigma $= " + f"{stats[2]:.2f} mmag"
df_m["DZPmmag"].hist(bins=NBINS,range=RANGE, histtype="step",color="b",ax=ax,alpha=1,legend=leg,label=label,linewidth=3)

histdata = df_m["DZPmmag"].values
counts,xedges =np.histogram(histdata,bins=NBINS,range=RANGE)
xcenters = (xedges[:-1] + xedges[1:]) / 2

# Fit the data using a Gaussian
g_init = models.Gaussian1D(amplitude=AMAX, mean=0, stddev=10.)
fit_g = fitting.TRFLSQFitter()
g0 = fit_g(g_init, xcenters, counts)
fit_m0 = g0.mean.value
fit_s0 = g0.stddev.value
   
yfit = g0(xfit)
ax.plot(xfit,yfit,"r-")
textstr = "\n".join((f"fit  of gaussian : ",
                     f"- mean : {fit_m0:.2f} mmag",
                     f"- sigma : {fit_s0:.2f} mmag",     
                    ))
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

ax.legend()
ax.legend(bbox_to_anchor=(1.01, 1.02),ncols=1,fontsize=12)
ax.set_title("zero-point over all the Focal Plane",fontsize=16)
ax.set_xlabel("$\Delta ZP$ (mmag)")
plt.suptitle(f"{instrument} (band {band}, mag < {MAGCUT} mag) \n {collectionStr}",fontsize=12)
figname =f"{pathfigs}/UNIFZPCOMBFP1D_1_{instrument}_{band}"+figtype
plt.savefig(figname)
plt.show()

#### All histogram of each detector shown on same plot, stat only

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6),layout="constrained",sharex=True)

for idet in range(NDET):
    leg = ax.get_legend()
    stats = df_m[df_m.detector==idet]["DZPmmag"].agg(["mean","median","std"])
    label = f"det = {idet} " + "$\overline{m}= $" + f" {stats[0]:.2f} mmag " + " $\sigma $= " + f"{stats[2]:.2f} mmag"
    df_m[df_m.detector==idet]["DZPmmag"].hist(bins=NBINS,range=RANGE, histtype="step",color=all_det_colors[idet],ax=ax,alpha=1,legend=leg,label=label,linewidth=3)
ax.legend()
ax.legend(bbox_to_anchor=(1.05, 1.02),ncols=1,fontsize=10)
ax.set_title("Zero-point over Focal Plane",fontsize=16)
ax.set_xlabel("$\Delta ZP$ (mmag)")
plt.suptitle(f"{instrument} (band {band}, mag < {MAGCUT} mag) \n {collectionStr}",fontsize=12)
figname =f"{pathfigs}/UNIFZPFP1D_1_{instrument}_{band}"+figtype
plt.savefig(figname,bbox_inches='tight')
plt.show()

In [ ]:
#from astropy.modeling import models, fitting
#props = dict(boxstyle='round', facecolor="white", alpha=0.1)

#### Individual histogram per detector stat and fit

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,16),layout="constrained",sharey=True)
axs = axes.flatten()
xfit = np.linspace(RANGE[0],RANGE[1],NBINS)
all_fitparams = {}
for idet, ax in enumerate(axs):
    leg = ax.get_legend()

    stats = df_m[df_m.detector==idet]["DZPmmag"].agg(["mean","median","std"])
    label = f"det = {idet} " + "$\overline{m}= $" + f" {stats[0]:.2f} mmag " + " $\sigma $= " + f"{stats[2]:.2f} mmag"

    # fit
    histdata = df_m[df_m.detector==idet]["DZPmmag"].values
    counts,xedges =np.histogram(histdata,bins=NBINS,range=RANGE)
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    # Fit the data using a Gaussian
    g_init = models.Gaussian1D(amplitude=AMAX, mean=0, stddev=10.)
    fit_g = fitting.TRFLSQFitter()
    g = fit_g(g_init, xcenters, counts)
    fit_m = g.mean.value
    fit_s = g.stddev.value
    all_fitparams[idet] = g
    yfit = g(xfit)
    textstr = "\n".join((f"fit  of gaussian : ",
                     f"- mean : {fit_m:.2f} mmag",
                     f"- sigma : {fit_s:.2f} mmag",     
                    ))
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

    
    df_m[df_m.detector==idet]["DZPmmag"].hist(bins=NBINS,range=RANGE, histtype="step",color=all_det_colors[idet],ax=ax,alpha=1,legend=leg,label=label,linewidth=3)
    ax.plot(xfit,yfit,'-k',lw=2)
    
    ax.set_aspect('auto')
    ax.set_title(f"det = {idet}")
    #ax.set_ylim(0.,0.5)
    ax.grid(True)
    ax.set_xlabel("$\Delta ZP$ (mmag)")
  
    
#plt.gca().set_aspect('equal')
#plt.suptitle(f"{instrument} (band {band}, mag < {MAGCUT} mag)\n {collectionStr}",fontsize=12)
plt.suptitle("$\Delta ZP$ (mmag) : " + f" {instrument} (band {band}, mag < {MAGCUT} mag)\n {collectionStr}",fontsize=14)
figname =f"{pathfigs}/UNIFZPFP1D_9_{instrument}_{band}"+figtype
plt.savefig(figname)
plt.show()

## Select a subset of TractID

In [ ]:
List_Of_Tracts = [2394, 4848, 4849, 5063, 5305, 5306, 5525, 5526, 10463]
NTRACT = len(List_Of_Tracts)
print(NTRACT) 

In [ ]:
cmap = ListedColormap(sns.color_palette("hls", NTRACT ))
all_tract_colors = [cmap.colors[idx] for idx in range(NTRACT )]

### Histogram of ZP per tract on one plot

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6),layout="constrained",sharex=True)

for itract,theTract in enumerate(List_Of_Tracts):
    leg = ax.get_legend()
    stats = df_m[df_m.tractID==theTract]["DZPmmag"].agg(["mean","median","std"])
    label = f"tract = {theTract} " + "$\overline{m}= $" + f" {stats[0]:.2f} mmag " + " $\sigma $= " + f"{stats[2]:.2f} mmag"
    df_m[df_m.tractID == theTract]["DZPmmag"].hist(bins=NBINS,range=RANGE, histtype="step",color=all_tract_colors[itract],ax=ax,alpha=1,legend=leg,label=label,linewidth=3)
ax.legend()
ax.legend(bbox_to_anchor=(1.05, 1.02),ncols=1,fontsize=10)
ax.set_title("Zero-point over Tract")
ax.set_xlabel("$\Delta ZP$ (mmag)")
plt.suptitle(f"{instrument} (band {band}, mag < {MAGCUT} mag) \n {collectionStr}",fontsize=12)
figname =f"{pathfigs}/UNIFZPTRACT1D_1_{instrument}_{band}"+figtype
plt.savefig(figname,bbox_inches='tight')
plt.show()

### Histogram of ZP per tract in different plots

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,16),layout="constrained")
axs = axes.flatten()

xfit = np.linspace(RANGE[0],RANGE[1],NBINS)
all_fitparams = {}
for itract, ax in enumerate(axs):
    theTractID = List_Of_Tracts[itract]
    leg = ax.get_legend()
    stats = df_m[df_m.tractID==theTractID]["DZPmmag"].agg(["mean","median","std"])
    label = f"tract = {theTractID} " + "$\overline{m}= $" + f" {stats[0]:.2f} mmag " + " $\sigma $= " + f"{stats[2]:.2f} mmag"

    # fit
    histdata = df_m[df_m.tractID==theTractID]["DZPmmag"].values
    counts,xedges =np.histogram(histdata,bins=NBINS,range=RANGE)
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    # Fit the data using a Gaussian
    g_init = models.Gaussian1D(amplitude=1000., mean=0, stddev=10.)
    fit_g = fitting.TRFLSQFitter()
    g = fit_g(g_init, xcenters, counts)
    fit_m = g.mean.value
    fit_s = g.stddev.value
    all_fitparams[theTractID] = g
    yfit = g(xfit)
    textstr = "\n".join((f"fit  of gaussian : ",
                     f"- mean : {fit_m:.2f} mmag",
                     f"- sigma : {fit_s:.2f} mmag",     
                    ))
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

    
    df_m[df_m.tractID==theTractID]["DZPmmag"].hist(bins=NBINS,range=RANGE, histtype="step",color=all_tract_colors[itract],ax=ax,alpha=1,legend=leg,label=label,linewidth=3)
    ax.plot(xfit,yfit,'-k',lw=2)
    
    ax.set_aspect('auto')
    ax.set_title(f"tract = {theTractID}")
    #ax.set_ylim(0.,0.5)
    ax.grid(True)
    ax.set_xlabel("$\Delta ZP$ (mmag)")
  
    
#plt.gca().set_aspect('equal')
#plt.suptitle(f"{instrument} (band {band}, mag < {MAGCUT} mag)\n {collectionStr}",fontsize=12)
plt.suptitle("$\Delta ZP$ (mmag) : " + f" {instrument} (band {band}, mag < {MAGCUT} mag)\n {collectionStr}",fontsize=14)
figname =f"{pathfigs}/UNIFZPTRACT1D_9_{instrument}_{band}"+figtype
plt.savefig(figname)
plt.show()

#### 2D Uniformity plot of $\Delta ZP$

In [ ]:
cmap = mpl.colormaps['seismic']
cmap_invert = mpl.colormaps['seismic_r']
norm = mpl.colors.Normalize(vmin=RANGE[0], vmax=RANGE[1])

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(18,18),layout="constrained")
axs = axes.flatten()

#for idet, ax in enumerate(axs):
for itract, ax in enumerate(axs):
    leg = ax.get_legend()
    
    theTractID = List_Of_Tracts[itract]
    leg = ax.get_legend()
    stats = df_m[df_m.tractID==theTractID]["DZPmmag"].agg(["mean","median","std"])
    label = f"tract = {theTractID} " + "$\overline{m}= $" + f" {stats[0]:.2f} mmag " + " $\sigma $= " + f"{stats[2]:.2f} mmag"

    
    fitparams = all_fitparams[theTractID]
    fit_m = fitparams.mean.value
    fit_s = fitparams.stddev.value
    label = f"tract = {theTractID} \n" + "$\overline{m}= $" + f" {fit_m:.2f} mmag \n" + " $\sigma $= " + f"{fit_s:.2f} mmag"
    df_data =  df_m[df_m.tractID==theTractID]
    rgba_color = cmap(norm(df_data["DZPmmag"].values )) 
    #df_data.plot.scatter(x="x",y="y",c="DZPmmag",cmap="seismic",ax=ax,marker="o",colorbar="True",vmin=-5,vmax=5.)
    #df_data.plot.scatter(x="x",y="y",s=1.,c=rgba_color,cmap=cm.seismic,ax=ax)
    im = ax.scatter("coord_ra", "coord_dec",c='DZPmmag',data=df_data,cmap="seismic",marker="o",label=label,vmin=RANGE[0],vmax=RANGE[1])
    ax.grid(True)
    ax.legend(fontsize=18)
    ax.set_aspect('equal')
plt.suptitle("$\Delta ZP$ (mmag) : " + f" {instrument} (band {band}, mag < {MAGCUT} mag)\n {collectionStr}",fontsize=16)

#fig.add_axes
#rect : tuple (left, bottom, width, height)
#    The dimensions (left, bottom, width, height) of the new
#    `~.axes.Axes`. All quantities are in fractions of figure width and
#    height.
cbar_ax = fig.add_axes([1.01, 0, 0.03, 1.0])

fig.colorbar(im, cax=cbar_ax)    
cbar_ax.set_ylabel('$\Delta ZP$ (mmag)', rotation=90)

figname =f"{pathfigs}/UNIFZPTRACT2D_9_{instrument}_{band}"+figtype
plt.savefig(figname,bbox_inches='tight')

plt.show()